## 1.2 Hidden Messages in the Replication Origin

Operating under the assumption that DNA is a language of its own, let's borrow Legrand's method and see if we can find any surprisingly frequent "words" within the ori of Vibrio cholerae. We have added reason to look for frequent words in the ori because for various biological processes, certain nucleotide strings often appear surprisingly often in small regions of the genome. This is often because certain proteins can only bind to DNA if a specific string of nucleotides is present, and if there are more occurrences of the string, then it is more likely that binding will successfully occur. (It is also less likely that a mutation will disrupt the binding process.)

For example, ACTAT is a surprisingly frequent substring of ACAACTATGCATACTATCGGGAACTATCCT.

**We will use the term k-mer to refer to a string of length k and define Count(Text, Pattern) as the number of times that a k-mer Pattern appears as a substring of Text. Following the above example**,

`Count(ACAACTATGCATACTATCGGGAACTATCCT, ACTAT) = 3`.

We note that Count(CGATATATCCATAG, ATA) is equal to 3 (not 2) since we should account for overlapping occurrences of Pattern in Text.

To compute Count(Text, Pattern), our plan is to “slide a window” down Text, checking whether each k-mer substring of Text matches Pattern. We will therefore refer to the k-mer starting at position i of Text as Text(i, k). Throughout this book, we will often use 0-based indexing, meaning that we count starting at 0 instead of 1. In this case, Text begins at position 0 and ends at position |Text| − 1 (|Text| denotes the number of symbols in Text). For example, if Text = GACCATACTG, then Text(4, 3) = ATA. Note that the last k-mer of Text begins at position |Text| − k, e.g., the last 3-mer of GACCATACTG starts at position 10 − 3 = 7. This discussion results in the following pseudocode for computing Count(Text, Pattern).

```
    PatternCount(Text, Pattern)
        count ← 0
        for i ← 0 to |Text| − |Pattern|
            if Text(i, |Pattern|) = Pattern
                count ← count + 1
        return count
```

Code Challenge: Implement PatternCount (reproduced below).
     Input: Strings Text and Pattern.
     Output: Count(Text, Pattern).

```
PatternCount(Text, Pattern)
  count ← 0
  for i ← 0 to |Text| − |Pattern|
    if Text(i, |Pattern|) = Pattern
      count ← count + 1
  return count
````

In [23]:
with open ("data/dataset_2_6.txt") as input_file:
    Text, Pattern = input_file.read().strip().split()

def PatternCount(Text,Pattern):
    count = 0
    for i in range(len(Text) - len(Pattern)):
        if Text[i:i+len(Pattern)] == Pattern:
            count += 1
    return count

total_count = PatternCount(Text,Pattern)

with open('output/dataset_2_6.txt', 'w') as output_data:
    output_data.write(str(total_count))

## The Frequent Words Problem

We say that Pattern is a most frequent k-mer in Text if it maximizes Count(Text, Pattern) among all k-mers. You can see that `ACTAT` is a most frequent 5-mer of `ACAACTATGCATACTATCGGGAACTATCCT`, and `ATA` is a most frequent 3-mer of CGATATATCCATAG.

STOP and Think: Can a string have multiple most frequent k-mers?

We now have a rigorously defined computational problem.

Frequent Words Problem: Find the most frequent k-mers in a string.

Input: A string Text and an integer k.
Output: All most frequent k-mers in Text.

 A straightforward algorithm for finding the most frequent k-mers in a string Text checks all k-mers appearing in this string (there are |Text| − k + 1 such k-mers) and then computes how many times each k-mer appears in Text. To implement this algorithm, called FrequentWords, we will need to generate an array Count, where Count(i) stores Count(Text, Pattern) for Pattern = Text(i, k) (see figure below).

The pseudocode for FrequentWords is shown below.

```
FrequentWords(Text, k)
    FrequentPatterns ← an empty set
    for i ← 0 to |Text| − k
        Pattern ← the k-mer Text(i, k)
        Count(i) ← PatternCount(Text, Pattern)
    maxCount ← maximum value in array Count
    for i ← 0 to |Text| − k
        if Count(i) = maxCount
            add Text(i, k) to FrequentPatterns
    remove duplicates from FrequentPatterns
    return FrequentPatterns
```

A Faster Frequent Words Approach:
If you were to solve the Frequent Words Problem by hand for a small example, you would probably form a table like the one in the figure below for Text equal to "ACGTTTCACGTTTTACGG" and k equal to 3. You would slide a length-k window Text, and if the current k-mer substring of text does not occur in the table, then you would create a new entry for it. Otherwise, you would add 1 to the entry corresponding to the current k-mer substring of Text. We call this table the frequency table for Text and k.

```
FrequencyTable(Text, k)
    freqMap ← empty map
    n ← |Text|
    for i ← 0 to n − k
        Pattern ← Text(i, k)
        if freqMap[Pattern] doesn't exist
            freqMap[Pattern]← 1
        else
           freqMap[pattern] ←freqMap[pattern]+1 
    return freqMap
```


In [30]:
def FrequencyTable(Text,k):
    freqMap = {}
    n = len(Text)
    for i in range(n-k):
        Pattern = Text[i:i+k]
        if Pattern not in freqMap:
            freqMap[Pattern] = 1
        else:
            freqMap[Pattern] += 1
    return freqMap

FrequencyTable('ACGTTGCATGTCGCATGATGCATGAGAGCT',4)

{'ACGT': 1,
 'CGTT': 1,
 'GTTG': 1,
 'TTGC': 1,
 'TGCA': 2,
 'GCAT': 3,
 'CATG': 3,
 'ATGT': 1,
 'TGTC': 1,
 'GTCG': 1,
 'TCGC': 1,
 'CGCA': 1,
 'ATGA': 2,
 'TGAT': 1,
 'GATG': 1,
 'ATGC': 1,
 'TGAG': 1,
 'GAGA': 1,
 'AGAG': 1,
 'GAGC': 1}

Once we have built the frequency table for a given Text and k, we can find all frequent k-mers if we determine the maximum value in the table, and then identify the keys of the frequency table achieving this value, appending each one that we find to a growing list. We are now ready to write a function BetterFrequentWords to solve the Frequent Words Problem.  This function relies on a function MaxMap that takes a map of strings to integers as an input and returns the maximum value of this map as output. 
```
BetterFrequentWords(Text, k)
    FrequentPatterns ← an array of strings of length 0
    freqMap ← FrequencyTable(Text, k)
    max ← MaxMap(freqMap)
    for all strings Pattern in freqMap
        if freqMap[pattern] = max
            append Pattern to frequentPatterns
    return frequentPatterns
```

In [51]:
with open ("data/dataset_2_13.txt") as input_file:
    Text, k = input_file.read().strip().split()
k = int(k)

def BetterFrequentWords(Text,k):
    FrequentPatterns = ""
    freqMap = FrequencyTable(Text, k)
    maxMap = max(freqMap.values())
    for Pattern in freqMap:
        if freqMap[Pattern] == maxMap:
            FrequentPatterns += Pattern + " "
    return FrequentPatterns

FrequentPatterns = BetterFrequentWords(Text,k)
print(FrequentPatterns)
with open('output/dataset_2_13.txt', 'w') as output_data:
    output_data.write((FrequentPatterns))

TGAATCTGTGAAT GAATCTGTGAATC AATCTGTGAATCT ATCTGTGAATCTG 


In [ ]:
with open ("data/Vibrio_cholerae.txt") as input_file:
    Text = input_file.read().strip()
    k = 2
print(Text)

FrequentPatterns = BetterFrequentWords(Text,k)
print(FrequentPatterns)
with open('output/Vibrio_cholerae.txt', 'w') as output_data:
    output_data.write((FrequentPatterns))